# Converting SQL from one system to another

Convert from MS SQL to PL/SQL and PostreSQL.

In [ ]:
--Example: Updating Inventory With Logging
CREATE PROCEDURE UpdateInventory
    @ProductID INT,
    @QuantityChange INT
AS
BEGIN
    BEGIN TRY
        BEGIN TRANSACTION
            -- Check if the product exists
            IF NOT EXISTS(SELECT 1 FROM Products WHERE ProductID = @ProductID)
                THROW 50000, 'Product does not exist.', 1;

            -- Update inventory
            UPDATE Products
            SET Quantity = Quantity + @QuantityChange
            WHERE ProductID = @ProductID;

            -- Log the inventory change
            INSERT INTO InventoryLog(ProductID, QuantityChange, ChangeDate)
            VALUES (@ProductID, @QuantityChange, GETDATE());

        COMMIT TRANSACTION
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        -- Return or log the error
        DECLARE @ErrorMessage NVARCHAR(4000) = ERROR_MESSAGE();
        THROW 50000, @ErrorMessage, 1;
    END CATCH
END